In [1]:
from google.cloud import bigquery
import pandas as pd
import plotly.express as px

In [2]:
key_path = "../credentials/dbt_service_account.json"
client = bigquery.Client.from_service_account_json(key_path)

In [5]:
query = """
SELECT
    COUNT(*) AS sessions,
    COUNTIF(has_added_to_cart) AS carts,
    COUNTIF(has_begun_checkout) AS checkouts,
    COUNTIF(has_purchased) AS purchases
FROM
    `amiable-dynamo-461319-g1.ga4_demo.fact_conversion_funnel`
    """

funnel = client.query(query).to_dataframe()

In [8]:
funnel

,sessions,carts,checkouts,purchases
0,241752,10651,8947,3733


In [13]:
stage_df = (
    funnel.iloc[0].rename({'sessions': 'Sessions', 'carts': 'Add-to-Cart', 'checkouts': 'Begin checkout', 'purchases': 'Purchase'})
    .to_frame("Count")
    .reset_index()
    .rename(columns = {'index': 'Stage'})
)

In [14]:
stage_df

,Stage,Count
0,Sessions,241752
1,Add-to-Cart,10651
2,Begin checkout,8947
3,Purchase,3733


In [16]:
fig = px.funnel(stage_df, x = "Count", y = "Stage", title = "Funnel (sessions)", text = "Count")
fig.show()